# Re-implementing procedure outlined in "Entity Profiling in Knowledge Graphs" (Zhang Et al.)
# This notebook will implement the candidate label creation step
Using a subset of wikidata related to Q44 ("beer")

In [72]:
import os

In [73]:
work_dir = "/Users/nicklein/Documents/grad_school/Research/data"

In [74]:
os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(work_dir)
os.environ['DATA'] = "{}/Q44".format(work_dir)
os.environ['NAME'] = "Q44"
os.environ["OUT"] = "{}/Q44_profiler_output".format(work_dir)

Helper function for executing commands

In [71]:
def run_command(command, substitution_dictionary = {}):
    """Run a templetized command."""
    #cmd = command.replace("NAME", subset_name)
    for k, v in substitution_dictionary.items():
        cmd = cmd.replace(k, v)
    
    print(cmd)
    output = subprocess.run([cmd], shell=True, universal_newlines=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(output.stdout)
    print(output.stderr)

## Examples of using query command:
### Ex - Counting instances of <type, property, value> along with the property label

In [131]:
!kgtk query -i $DATA/$NAME.part.wikibase-item.tsv -i $DATA/$NAME.label.en.tsv --graph-cache $STORE \
--match 'item: (n1)-[l {label: p}]->(n2), item: (n1)-[:P31]->(type), label: (p)-[:label]->(lab)' \
--return 'distinct type as type, l.label as prop, n2 as value, count(n1) as count, lab as property_label' \
--where 'lab.kgtk_lqstring_lang_suffix = "en"' \
--order-by 'type, count(n1) desc' \
--limit 5

type	prop	value	count	property_label
Q1066984	P1343	Q97879676	3	'described by source'@en
Q1066984	P1343	Q316838	2	'described by source'@en
Q1066984	P17	Q154195	2	'country'@en
Q1066984	P17	Q183	2	'country'@en
Q1066984	P17	Q2415901	2	'country'@en


### Ex - Counting property, value pairs for time properties

In [165]:
!kgtk query -i $DATA/$NAME.part.time.tsv -i $DATA/$NAME.label.en.tsv --graph-cache $STORE \
--match 'time: (n1)-[l {label: p}]->(n2), label: (p)-[:label]->(lab)' \
--return 'distinct l.label as prop, lab as property_label, kgtk_date_year(n2) as val, count(n1) as count' \
--where 'lab.kgtk_lqstring_lang_suffix = "en"' \
--order-by 'count(n1) desc' \
--limit 5

prop	property_label	val	count
P571	'inception'@en	1991	20
P571	'inception'@en	1918	14
P571	'inception'@en	1989	8
P571	'inception'@en	1996	8
P571	'inception'@en	1821	6


### Ex- Counting property, value pairs for quantity properties

In [133]:
!kgtk query -i $DATA/$NAME.part.quantity.tsv -i $DATA/$NAME.label.en.tsv --graph-cache $STORE \
--match 'quantity: (n1)-[l {label: p}]->(n2), label: (p)-[:label]->(lab)' \
--return 'distinct l.label as prop, lab as property_label, n2 as value, count(n1) as count' \
--where 'lab.kgtk_lqstring_lang_suffix = "en"' \
--order-by 'count(n1) desc' \
--limit 5

prop	property_label	value	count
P3000	'marriageable age'@en	+18Q24564698	56
P2997	'age of majority'@en	+18Q24564698	52
P1279	'inflation rate'@en	+1.7Q11229	41
P2884	'mains voltage'@en	+230Q25250	40
P1279	'inflation rate'@en	+1.8Q11229	38


# Outline of procedure:
**Goal**:<br>
We want to create candidate label sets including
- Attribute value labels (type, property, *attribute*)
- Realtional entity labels (type, property, *entity*)
- Attribute interval labels (type, property, *range of attribute values*)
- Relational attribute labels (type, property, *attribute or attribute range of another entity*)

To enable subsequent filtering of these labels, we also want to count:
- The number of entities of each type
- The number of entities that match each label (call these "positives")
    
**Steps**:

0. Create type-mapping
1. Count the number of entities of each type
    - *optional future step*: define type with P279 transitive closure in addition to P31. 
2. Create AVLs trivially from attribute files along with counts of the positive entities for each label
    - At this step, we should also contribute to a mapping of entities --> matching attribute labels to facilitate creating RALs in a later step  
3. Create RELs trivially from entity relation files along with counts of positive entities for each label
4. Create AILs by discretizing the AVLs we found, along with counts of positive entities for each label
    - This can be done using the counts that we already have
    - We'll likely try a naive approach at first such as hardcoding a bucket size for each numeric type we care about
    - At this step, we should also contribute to a mapping of entities --> matching attribute labels to facilitate creating RALs in a later step
5. Create RALs by using the REL table and the entities --> attribute labels table that built in steps 2 and 4. Also keep track of counts of positive entities for each label
    
*Misc issues encountered*
- kgtk rename-columns doesn't always work when input file == output file. Getting around this right now by creating extra temp files... 

## 0. Create type mapping
Using P31 only for now, but can add P279* as well later

In [361]:
!kgtk filter -p ' ; P31 ; ' -i $DATA/$NAME.part.wikibase-item.tsv -o $OUT/$NAME.type_mapping.tsv

In [638]:
!head -5 $OUT/$NAME.type_mapping.tsv | column -t -s $'\t'

id              node1     label  node2
Q1000597-P31-1  Q1000597  P31    Q3957
Q1011-P31-2     Q1011     P31    Q112099
Q1011-P31-1     Q1011     P31    Q3624078
Q1019-P31-2     Q1019     P31    Q112099


## 1. Count number of entities of each type:

In [589]:
!kgtk query -i $OUT/$NAME.type_mapping.tsv -i $DATA/$NAME.label.en.tsv -o $OUT/$NAME.entity_counts_per_type_temp.tsv --graph-cache $STORE \
--match 'type: (n1)-[]->(type), label: (type)-[:label]->(lab)' \
--return 'distinct type as type, lab as type_label, count(distinct n1) as count, "_" as id' \
--where 'lab.kgtk_lqstring_lang_suffix = "en"' \
--order-by 'count(distinct n1) desc' \

In [590]:
!kgtk rename-columns -i $OUT/$NAME.entity_counts_per_type_temp.tsv -o $OUT/$NAME.entity_counts_per_type_temp1.tsv \
--old-columns type type_label count --new-columns node1 label node2 

In [591]:
!kgtk add-id -i $OUT/$NAME.entity_counts_per_type_temp1.tsv \
-o $OUT/$NAME.entity_counts_per_type.tsv --overwrite-id

In [637]:
!head -5 $OUT/$NAME.entity_counts_per_type.tsv | column -t -s $'\t'

node1     label                 node2  id
Q131734   'brewery'@en          87     E1
Q3624078  'sovereign state'@en  69     E2
Q4830453  'business'@en         50     E3
Q6256     'country'@en          26     E4


## 2. Create AVLs with counts of positive entities
At this step we also want to keep track of entities --> matching attribute labels for future use. This will help when we are creating RALs (step 5)

We should also keep numeric valued labels and non-numeric valued labels separate to help with discretization at a later step

### strings
Creating mapping of entity --> string attribute labels

In [728]:
!kgtk query -i $OUT/$NAME.type_mapping.tsv -i $DATA/$NAME.part.string.tsv -i $DATA/$NAME.label.en.tsv \
-o $OUT/$NAME.entity_attribute_labels_string_temp.tsv --graph-cache $STORE \
--match 'string: (n1)-[l {label:p}]->(n2), type: (n1)-[]->(type), label: (p)-[:label]->(lab)' \
--return 'distinct n1 as entity, type as type, p as prop, n2 as value, lab as property_label, "_" as id' \
--where 'lab.kgtk_lqstring_lang_suffix = "en"' \
--order-by 'n1'

In [729]:
!kgtk rename-columns -i $OUT/$NAME.entity_attribute_labels_string_temp.tsv \
-o $OUT/$NAME.entity_attribute_labels_string_temp1.tsv \
--old-columns type prop value --new-columns node1 label node2

In [730]:
!kgtk add-id -i $OUT/$NAME.entity_attribute_labels_string_temp1.tsv \
-o $OUT/$NAME.entity_attribute_labels_string.tsv --overwrite-id

In [731]:
!head -5 $OUT/$NAME.entity_attribute_labels_string.tsv | column -t -s $'\t'

entity    node1  label  node2                property_label           id
Q1000597  Q3957  P281   "DE14"               'postal code'@en         E1
Q1000597  Q3957  P373   "Burton upon Trent"  'Commons category'@en    E2
Q1000597  Q3957  P473   "01283"              'local dialing code'@en  E3
Q1000597  Q3957  P613   "SK245225"           'OS grid reference'@en   E4


Aggregating distinct labels w/ positive entity counts

In [732]:
!kgtk query -i $OUT/$NAME.entity_attribute_labels_string.tsv \
-o $OUT/$NAME.candidate_labels_avl_string_temp.tsv --graph-cache $STORE \
--match 'labels: (type)-[l {label:prop, property_label:lab, entity:e}]->(val)' \
--return 'distinct type as type, prop as prop, val as value, count(distinct e) as positives, lab as property_label, "_" as id' \
--order-by 'count(distinct e) desc'

In [733]:
!kgtk rename-columns -i $OUT/$NAME.candidate_labels_avl_string_temp.tsv \
-o $OUT/$NAME.candidate_labels_avl_string_temp1.tsv \
--old-columns type prop value --new-columns node1 label node2

In [734]:
!kgtk remove-columns -c "id" -i $OUT/$NAME.candidate_labels_avl_string_temp1.tsv \
-o $OUT/$NAME.candidate_labels_avl_string_temp2.tsv

In [735]:
!kgtk add-id -i $OUT/$NAME.candidate_labels_avl_string_temp2.tsv -o $OUT/$NAME.candidate_labels_avl_string.tsv

In [736]:
!head -5 $OUT/$NAME.candidate_labels_avl_string.tsv | column -t -s $'\t'

node1     label  node2    positives  property_label     id
Q3624078  P3238  "0"      34         'trunk prefix'@en  E1
Q3624078  P3238  novalue  14         'trunk prefix'@en  E2
Q6256     P3238  "0"      12         'trunk prefix'@en  E3
Q179164   P3238  "0"      9          'trunk prefix'@en  E4


### Times

Looking at what precisions we need to deal with...

In [634]:
!kgtk query -i $DATA/$NAME.part.time.tsv $DATA/$NAME.label.en.tsv\
--graph-cache $STORE \
--match 'time: (n1)-[l {label:p}]->(n2), label: (p)-[:label]->(lab)' \
--return 'distinct kgtk_date_precision(n2) as precisions, count(n1) as count' \
--limit 10 \
| column -t -s $'\t'

precisions  count
6           12
7           40
8           12
9           697
10          48
11          469


From the above, we have several precisions below precision of year=9. We don't have kgtk type interpretation functions for these granularities, so for now we'll interpret them all as years. Furthermore, we will interpret all times at the year granularity for now.

Additional work can be done later to create labels with finer time granularity if desired.

#### precision = year or broader
#### note - at this time we will include values that have precision finer than a year here as well
We interpret these as years

In [874]:
!kgtk query -i $OUT/$NAME.type_mapping.tsv -i $DATA/$NAME.part.time.tsv -i $DATA/$NAME.label.en.tsv \
-o $OUT/$NAME.entity_attribute_labels_time.year_temp.tsv --graph-cache $STORE \
--match 'time: (n1)-[l {label:p}]->(n2), type: (n1)-[]->(type), label: (p)-[:label]->(p_lab), label: (type)-[:label]->(t_lab)' \
--return 'distinct n1 as entity, type as type, p as prop, kgtk_date_year(n2) as value, t_lab as type_label, p_lab as property_label, "_" as id' \
--where 't_lab.kgtk_lqstring_lang_suffix = "en" AND p_lab.kgtk_lqstring_lang_suffix = "en"' \
--order-by 'n1'

In [875]:
!kgtk rename-columns -i $OUT/$NAME.entity_attribute_labels_time.year_temp.tsv \
-o $OUT/$NAME.entity_attribute_labels_time.year_temp1.tsv \
--old-columns type prop value --new-columns node1 label node2

In [876]:
!kgtk remove-columns -c "id" -i $OUT/$NAME.entity_attribute_labels_time.year_temp1.tsv \
-o $OUT/$NAME.entity_attribute_labels_time.year_temp2.tsv

In [877]:
!kgtk add-id -i $OUT/$NAME.entity_attribute_labels_time.year_temp2.tsv \
-o $OUT/$NAME.entity_attribute_labels_time.year.tsv

In [878]:
!head -5 $OUT/$NAME.entity_attribute_labels_time.year.tsv | column -t -s $'\t'

entity  node1     label  node2  type_label            property_label  id
Q1011   Q112099   P571   1975   'island nation'@en    'inception'@en  E1
Q1011   Q3624078  P571   1975   'sovereign state'@en  'inception'@en  E2
Q1019   Q112099   P571   1960   'island nation'@en    'inception'@en  E3
Q1019   Q3624078  P571   1960   'sovereign state'@en  'inception'@en  E4


Aggregating distinct labels w/ positive entity counts

In [879]:
!kgtk query -i $OUT/$NAME.entity_attribute_labels_time.year.tsv \
-o $OUT/$NAME.candidate_labels_avl_time.year_temp.tsv --graph-cache $STORE \
--match 'labels: (n1)-[l {label:p, property_label:lab, entity:e}]->(val)' \
--return 'distinct n1 as type, p as prop, val as value, count(distinct e) as positives, lab as property_label, "_" as id' \
--order-by 'count(distinct e) desc'

In [880]:
!kgtk rename-columns -i $OUT/$NAME.candidate_labels_avl_time.year_temp.tsv \
-o $OUT/$NAME.candidate_labels_avl_time.year_temp1.tsv \
--old-columns type prop value --new-columns node1 label node2

In [881]:
!kgtk remove-columns -c "id" -i $OUT/$NAME.candidate_labels_avl_time.year_temp1.tsv \
-o $OUT/$NAME.candidate_labels_avl_time.year_temp2.tsv

In [882]:
!kgtk add-id -i $OUT/$NAME.candidate_labels_avl_time.year_temp2.tsv \
-o $OUT/$NAME.candidate_labels_avl_time.year.tsv

In [883]:
!head -5 $OUT/$NAME.candidate_labels_avl_time.year.tsv | column -t -s $'\t'

node1     label  node2  positives  property_label  id
Q3624078  P571   1991   8          'inception'@en  E1
Q3624078  P571   1918   7          'inception'@en  E2
Q6256     P571   1918   5          'inception'@en  E3
Q6256     P571   1991   5          'inception'@en  E4


#### ~~precision finer than a year~~
~~we interpret these as dates with day, month, year~~

Note, at this time we will give all time values a granularity of year. The results of the below code are currently not used.

In [737]:
!kgtk query -i $OUT/$NAME.type_mapping.tsv -i $DATA/$NAME.part.time.tsv -i $DATA/$NAME.label.en.tsv \
-o $OUT/$NAME.entity_attribute_labels_time.date_temp.tsv --graph-cache $STORE \
--match 'time: (n1)-[l {label:p}]->(n2), type: (n1)-[]->(type), label: (p)-[:label]->(lab)' \
--return 'distinct n1 as entity, type as type, p as prop, kgtk_date_date(n2) as value, lab as property_label, "_" as id' \
--where 'lab.kgtk_lqstring_lang_suffix = "en" AND kgtk_date_precision(n2) > 9' \
--order-by 'n1'

In [738]:
!kgtk rename-columns -i $OUT/$NAME.entity_attribute_labels_time.date_temp.tsv \
-o $OUT/$NAME.entity_attribute_labels_time.date_temp1.tsv \
--old-columns type prop value --new-columns node1 label node2

In [739]:
!kgtk remove-columns -c "id" -i $OUT/$NAME.entity_attribute_labels_time.date_temp1.tsv \
-o $OUT/$NAME.entity_attribute_labels_time.date_temp2.tsv

In [740]:
!kgtk add-id -i $OUT/$NAME.entity_attribute_labels_time.date_temp2.tsv \
-o $OUT/$NAME.entity_attribute_labels_time.date.tsv

In [741]:
!head -5 $OUT/$NAME.entity_attribute_labels_time.date.tsv | column -t -s $'\t'

entity    node1     label  node2        property_label  id
Q1011     Q112099   P571   ^1975-07-05  'inception'@en  E1
Q1011     Q3624078  P571   ^1975-07-05  'inception'@en  E2
Q1020773  Q3957     P571   ^1892-10-12  'inception'@en  E3
Q1020773  Q902814   P571   ^1892-10-12  'inception'@en  E4


Aggregating distinct labels w/ positive entity counts

In [742]:
!kgtk query -i $OUT/$NAME.entity_attribute_labels_time.date.tsv \
-o $OUT/$NAME.candidate_labels_avl_time.date_temp.tsv --graph-cache $STORE \
--match 'labels: (n1)-[l {label:p, property_label:lab, entity:e}]->(val)' \
--return 'distinct n1 as type, p as prop, val as value, count(distinct e) as positives, lab as property_label, "_" as id' \
--order-by 'count(distinct e) desc'

In [743]:
!kgtk rename-columns -i $OUT/$NAME.candidate_labels_avl_time.date_temp.tsv \
-o $OUT/$NAME.candidate_labels_avl_time.date_temp1.tsv \
--old-columns type prop value --new-columns node1 label node2

In [744]:
!kgtk remove-columns -c "id" -i $OUT/$NAME.candidate_labels_avl_time.date_temp1.tsv \
-o $OUT/$NAME.candidate_labels_avl_time.date_temp2.tsv

In [745]:
!kgtk add-id -i $OUT/$NAME.candidate_labels_avl_time.date_temp2.tsv \
-o $OUT/$NAME.candidate_labels_avl_time.date.tsv

In [746]:
!head -5 $OUT/$NAME.candidate_labels_avl_time.date.tsv | column -t -s $'\t'

node1     label  node2         positives  property_label                        id
Q1066984  P1249  ^1158-06-14   1          'time of earliest written record'@en  E1
Q1093829  P571   ^1790-07-16   1          'inception'@en                        E2
Q112099   P571   ^-0660-02-11  1          'inception'@en                        E3
Q112099   P571   ^1898-12-10   1          'inception'@en                        E4


## Quantities
Not using a kgtk literal function for now so we can keep the units attached to the values. Otherwise, could interpret everything as a number and drop the units. Seems like we need the units though.

In [918]:
!kgtk query -i $OUT/$NAME.type_mapping.tsv -i $DATA/$NAME.part.quantity.tsv -i $DATA/$NAME.label.en.tsv \
-o $OUT/$NAME.entity_attribute_labels_quantity_temp.tsv --graph-cache $STORE \
--match 'quantity: (n1)-[l {label:p}]->(n2), type: (n1)-[]->(type), label: (p)-[:label]->(p_lab), label: (type)-[:label]->(t_lab)' \
--return 'distinct n1 as entity, type as type, p as prop, kgtk_quantity_number(n2) as value, kgtk_quantity_si_units(n2) as si_units, kgtk_quantity_wd_units(n2) as wd_units, t_lab as type_label, p_lab as property_label, "_" as id' \
--where 't_lab.kgtk_lqstring_lang_suffix = "en" AND p_lab.kgtk_lqstring_lang_suffix = "en"' \
--order-by 'n1'

In [919]:
!kgtk rename-columns -i $OUT/$NAME.entity_attribute_labels_quantity_temp.tsv \
-o $OUT/$NAME.entity_attribute_labels_quantity_temp1.tsv \
--old-columns type prop value --new-columns node1 label node2

In [920]:
!kgtk remove-columns -c "id" -i $OUT/$NAME.entity_attribute_labels_quantity_temp1.tsv \
-o $OUT/$NAME.entity_attribute_labels_quantity_temp2.tsv

In [921]:
!kgtk add-id -i $OUT/$NAME.entity_attribute_labels_quantity_temp2.tsv \
-o $OUT/$NAME.entity_attribute_labels_quantity.tsv

In [922]:
!head -5 $OUT/$NAME.entity_attribute_labels_quantity.tsv | column -t -s $'\t'

entity    node1    label  node2  si_units            wd_units                      type_label  property_label  id
Q1000597  Q3957    P1082  75074  'town'@en           'population'@en               E1
Q1011     Q112099  P1081  0.57   'island nation'@en  'Human Development Index'@en  E2
Q1011     Q112099  P1081  0.572  'island nation'@en  'Human Development Index'@en  E3
Q1011     Q112099  P1081  0.575  'island nation'@en  'Human Development Index'@en  E4


Aggregating distinct labels w/ positive entity counts

In [752]:
!kgtk query -i $OUT/$NAME.entity_attribute_labels_quantity.tsv \
-o $OUT/$NAME.candidate_labels_avl_quantity_temp.tsv --graph-cache $STORE \
--match 'labels: (n1)-[l {label:p, property_label:lab, entity:e}]->(val)' \
--return 'distinct n1 as type, p as prop, val as value, count(distinct e) as positives, lab as property_label, "_" as id' \
--order-by 'count(distinct e) desc'

In [753]:
!kgtk rename-columns -i $OUT/$NAME.candidate_labels_avl_quantity_temp.tsv \
-o $OUT/$NAME.candidate_labels_avl_quantity_temp1.tsv \
--old-columns type prop value --new-columns node1 label node2

In [754]:
!kgtk remove-columns -c "id" -i $OUT/$NAME.candidate_labels_avl_quantity_temp1.tsv \
-o $OUT/$NAME.candidate_labels_avl_quantity_temp2.tsv

In [755]:
!kgtk add-id -i $OUT/$NAME.candidate_labels_avl_quantity_temp2.tsv \
-o $OUT/$NAME.candidate_labels_avl_quantity.tsv

In [756]:
!head -5 $OUT/$NAME.candidate_labels_avl_quantity.tsv | column -t -s $'\t'

node1     label  node2         positives  property_label         id
Q3624078  P3000  +18Q24564698  54         'marriageable age'@en  E1
Q3624078  P2997  +18Q24564698  51         'age of majority'@en   E2
Q3624078  P2884  +230Q25250    38         'mains voltage'@en     E3
Q3624078  P1279  +1.7Q11229    25         'inflation rate'@en    E4


### Combining entity --> attribute label mappings to single table

Note, currently omitting OUT/NAME.entity_attribute_labels_time.date.tsv from below

In [172]:
!kgtk cat \
-i $OUT/$NAME.entity_attribute_labels_string.tsv \
-i $OUT/$NAME.entity_attribute_labels_time.year.tsv \
-i $OUT/$NAME.entity_attribute_labels_quantity.tsv \
-o $OUT/$NAME.entity_AVLs_all.tsv

## 3. Create RELs with counts of positive entities

In [478]:
!kgtk query -i $DATA/$NAME.part.wikibase-item.tsv -i $OUT/$NAME.type_mapping.tsv -i $DATA/$NAME.label.en.tsv \
-o $OUT/$NAME.candidate_labels_rel_item_temp.tsv --graph-cache $STORE \
--match 'item: (n1)-[l {label:p}]->(n2), type: (n1)-[]->(type), label: (p)-[:label]->(lab)' \
--return 'distinct type as type, p as prop, n2 as value, count(distinct n1) as positives, lab as property_label, "_" as id' \
--where 'lab.kgtk_lqstring_lang_suffix = "en"' \
--order-by 'count(distinct n1) desc'

In [479]:
!kgtk rename-columns -i $OUT/$NAME.candidate_labels_rel_item_temp.tsv \
-o $OUT/$NAME.candidate_labels_rel_item_temp1.tsv \
--old-columns type prop value --new-columns node1 label node2

In [480]:
!kgtk remove-columns -c "id" -i $OUT/$NAME.candidate_labels_rel_item_temp1.tsv \
-o $OUT/$NAME.candidate_labels_rel_item_temp2.tsv

In [481]:
!kgtk add-id -i $OUT/$NAME.candidate_labels_rel_item_temp2.tsv \
-o $OUT/$NAME.candidate_labels_rel_item.tsv

In [627]:
!head -5 $OUT/$NAME.candidate_labels_rel_item.tsv | column -t -s $'\t'

node1     label  node2    positives  property_label            id
Q131734   P452   Q869095  77         'industry'@en             E1
Q3624078  P463   Q1065    68         'member of'@en            E2
Q3624078  P463   Q7817    67         'member of'@en            E3
Q3624078  P530   Q183     67         'diplomatic relation'@en  E4


## 4. Create AILs with counts of positive entities
### 4.a. Keep track of entities --> matching attribute labels for future use


In [141]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from kneed import KneeLocator


# Chooses values for the epsilon and min_samples parameters of DBSCAN to be used for clustering given labels.
def choose_DBSCAN_params(labels_df):
    values = np.array(labels_df.loc[:,"node2"]).reshape(-1, 1)
    
    # edge case only one sample will give an error if we try to compute knn.
    if len(values) < 2:
        print("only one sample for label:\n{}\n".format(labels_df))
        return (1, 1)
    
    # min_samples would ideally be set with some domain-insight.
    # To make this automated, we'll use a heuristic - ln(number of data points)
    min_samples = int(np.floor(np.log(len(values))))
    min_samples = max(min_samples, 1) # don't choose a number less than 1
    
    # epsilon can be chosen by plotting the distances of the k'th nearest neighbor from each point
    # where k is min_samples. Points that belong to clusters should have smaller distances, whereas
    # noise points can have distances that are much farther. We'll look for a knee in this graph to set epsilon.
    neigh = NearestNeighbors(n_neighbors = min_samples + 1) # +1 so we find k'th nearest neighbor not including the point iteslf.
    values_neigh = neigh.fit(values)
    distances, indices = values_neigh.kneighbors(values)
    distances = np.sort(distances[:,min_samples], axis = 0)
    
    kneedle = KneeLocator(range(len(distances)), distances, S=1, curve='convex', direction='increasing', interp_method = 'polynomial')
    epsilon = kneedle.knee_y
    if epsilon == None:
        print("no knee found for these labels:\n{}".format(labels_df))
        epsilon = distances[len(distances) // 2]
        print("using median distance to k nearest neighbor instead ({})\n".format(epsilon))
    
    return (min_samples, epsilon)


# Uses DBSCAN to assign values to clusters
def DBSCAN_1d_values(values, eps, min_samples):
    # dbscan doesn't like eps == 0. This translates to not doing any bucketing.
    # handling this as an edge case so dbscan doesn't throw an exception
    if eps <= 0:
        return values # same-valued points are labeled the same
    
    # dbscan expects multiple dimensions. Edit new object instead of existing view of df.
    values = np.array(values)
    values = np.append(values.reshape(-1,1), np.zeros((len(values),1)), axis = 1)
        
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(values)

    return (db.labels_)


# Given values that have been assigned to clusters,
# return a list of intervals that are consistent with these clusters
def get_intervals_for_values_based_on_clusters(values, labels):
    
    values = np.array(values)
    labels = np.array(labels)
    indexes = np.arange(len(values))
    
    # sort values and corresponding labels in ascending order
    labels = np.array([l for l, v in sorted(zip(labels, values), key=lambda pair: pair[1])])
    indexes = np.array([i for i, v in sorted(zip(indexes, values), key=lambda pair: pair[1])])
    values.sort()
    
    intervals = [(None, None)] # initially a single interval with no lower or upper bound
    
    # create intervals
    cur_label = labels[0]
    for i in range(len(labels)):
        # if new label, set upper bound of previous interval,
        # and start lower bound of a new interval.
        if labels[i] != cur_label:
            prev_interval_lb = (intervals[-1])[0]
            new_interval_edge = values[i-1] + ((values[i] - values[i-1]) / 2)
            intervals[-1] = (prev_interval_lb, new_interval_edge)
            intervals.append((new_interval_edge, None))
            cur_label = labels[i]
    
    # assign intervals to values
    cur_label = labels[0]
    cur_interval_ix = 0
    intervals_for_values = []
    for i in range(len(labels)):
        if labels[i] != cur_label:
            cur_interval_ix += 1
            cur_label = labels[i]
        intervals_for_values.append(intervals[cur_interval_ix])
    
    # rearrange intervals to original order of values
    intervals_for_values_unscrambled = np.zeros(len(intervals_for_values), dtype=tuple)
    for i in range(len(indexes)):
        intervals_for_values_unscrambled[indexes[i]] = intervals_for_values[i]
    
    return intervals_for_values_unscrambled

# Given a file containing numeric valued attribute labels,
# discretize these labels create a new file with the resulting attribute interval labels
def discretize_labels(avl_file_in, ail_file_out):
    df = pd.read_csv(avl_file_in, delimiter='\t')
    
    # add lower bound and upper bound columns
    df.insert(loc = len(df.columns), column = "lower_bound", value = ["" for i in range(df.shape[0])])
    df.insert(loc = len(df.columns), column = "upper_bound", value = ["" for i in range(df.shape[0])])
    
    # blank values in units columns are expected as not all values will have units.
    # we want blank units to compare equal to eachother, so fill NaN's in as "" in units columns.
    if "si_units" in df.columns and "wd_units" in df.columns:
        df.fillna("", inplace = True)
        
    # we also don't want to consider any string type values
    types = [type(v) for v in df.loc[:,"node2"]]
    non_str_mask = [True if (t != str) else False for t in types]
    df = df.loc[non_str_mask]
    
    # get distinct label types (defined by type and property, as well as si and wd units if we have them)
    if "si_units" in df.columns and "wd_units" in df.columns:
        distinct_labels = df.loc[:, ["node1", "label", "si_units", "wd_units"]].drop_duplicates()
    else:
        distinct_labels = df.loc[:, ["node1", "label"]].drop_duplicates()
        
    # Could probably be improved with a list comprehension
    for index, row in distinct_labels.iterrows():
        # Get subset of labels that match this distinct kind of label
        subset_mask = (df["node1"] == row["node1"]) & (df["label"] == row["label"])
        # if we have units, treat these as part of the kind of label
        if "si_units" in df.columns and "wd_units" in df.columns:
            subset_mask = subset_mask & (df["si_units"] == row["si_units"]) & (df["wd_units"] == row["wd_units"])
        subset = df.loc[subset_mask]
        
        values = subset.loc[:,"node2"]
        if(len(values) == 0):
            print("no values found for subset:\n{}\n".format(subset))
            print("row:\n{}\n".format(row))
        
        min_samples, epsilon = choose_DBSCAN_params(subset)
        cluster_labels = DBSCAN_1d_values(values, epsilon, min_samples)
        intervals_for_values = get_intervals_for_values_based_on_clusters(values, cluster_labels)
        lbounds = [pair[0] for pair in intervals_for_values]
        ubounds = [pair[1] for pair in intervals_for_values]
        df.loc[subset_mask,"lower_bound"] = lbounds
        df.loc[subset_mask,"upper_bound"] = ubounds
    
    df.to_csv(ail_file_out, sep = '\t', index = False)

In [143]:
avl_file_in = "{}/{}.entity_attribute_labels_time.year.tsv".format(os.environ["OUT"], os.environ["NAME"])
ail_file_out = "{}/{}.entity_attribute_labels_time.year_bucketed.tsv".format(os.environ["OUT"], os.environ["NAME"])
discretize_labels(avl_file_in, ail_file_out)

only one sample for label:
     entity  node1 label  node2 type_label  property_label  id lower_bound  \
4  Q1020773  Q3957  P571   1892  'town'@en  'inception'@en  E5               

  upper_bound  
4              

only one sample for label:
     entity    node1 label  node2        type_label  property_label  id  \
5  Q1020773  Q902814  P571   1892  'border town'@en  'inception'@en  E6   

  lower_bound upper_bound  
5                          

only one sample for label:
  entity     node1 label  node2                type_label  property_label  id  \
7  Q1027  Q2221906  P571   1968  'geographic location'@en  'inception'@en  E8   

  lower_bound upper_bound  
7                          

only one sample for label:
  entity     node1 label  node2                   type_label  property_label  \
9  Q1027  Q4198907  P571   1968  'parliamentary republic'@en  'inception'@en   

    id lower_bound upper_bound  
9  E10                          

no knee found for these labels:
    entity    

/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/kneed/knee_locator.py:304: UserWarning: No knee/elbow found
  warnings.warn("No knee/elbow found")
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/kneed/knee_locator.py:304: UserWarning: No knee/elbow found
  warnings.warn("No knee/elbow found")
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/pytho

only one sample for label:
    entity     node1 label  node2            type_label  \
82  Q15180  Q3624078  P576   1991  'sovereign state'@en   

                             property_label   id lower_bound upper_bound  
82  'dissolved, abolished or demolished'@en  E83                          

only one sample for label:
    entity    node1 label  node2             type_label  \
83  Q15180  Q512187  P576   1991  'federal republic'@en   

                             property_label   id lower_bound upper_bound  
83  'dissolved, abolished or demolished'@en  E84                          

only one sample for label:
    entity    node1 label  node2            type_label  \
84  Q15180  Q842112  P576   1991  'socialist state'@en   

                             property_label   id lower_bound upper_bound  
84  'dissolved, abolished or demolished'@en  E85                          

only one sample for label:
    entity    node1 label  node2            type_label  \
85  Q15180  Q849866  P576 

/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/kneed/knee_locator.py:218: RuntimeWarning: invalid value encountered in true_divide
  return (a - min(a)) / (max(a) - min(a))
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/kneed/knee_locator.py:242: RuntimeWarning: No local maxima found in the difference curve
The line is probably not polynomial, try plotting
the difference curve with plt.plot(knee.x_difference, knee.y_difference)
Also check that you aren't mistakenly setting the curve argument
  RuntimeWarning,
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/kneed/knee_locator

only one sample for label:
    entity     node1  label  node2         type_label  \
149  Q1726  Q1187811  P1249   1158  'college town'@en   

                           property_label    id lower_bound upper_bound  
149  'time of earliest written record'@en  E150                          

only one sample for label:
    entity    node1  label  node2             type_label  \
150  Q1726  Q134626  P1249   1158  'district capital'@en   

                           property_label    id lower_bound upper_bound  
150  'time of earliest written record'@en  E151                          

only one sample for label:
    entity      node1  label  node2                     type_label  \
151  Q1726  Q14784328  P1249   1158  'state capital in Germany'@en   

                           property_label    id lower_bound upper_bound  
151  'time of earliest written record'@en  E152                          

only one sample for label:
    entity     node1  label  node2                              type

/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/kneed/knee_locator.py:304: UserWarning: No knee/elbow found
  warnings.warn("No knee/elbow found")
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/kneed/knee_locator.py:304: UserWarning: No knee/elbow found
  warnings.warn("No knee/elbow found")
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/pytho

only one sample for label:
    entity    node1  label  node2               type_label  \
230   Q213  Q123480  P1249    900  'landlocked country'@en   

                           property_label    id lower_bound upper_bound  
230  'time of earliest written record'@en  E231                          

no knee found for these labels:
    entity  node1  label  node2    type_label  \
232   Q213  Q6256  P1249    900  'country'@en   
378    Q34  Q6256  P1249   1100  'country'@en   

                           property_label    id lower_bound upper_bound  
232  'time of earliest written record'@en  E233                          
378  'time of earliest written record'@en  E379                          
using median distance to k nearest neighbor instead (200.0)

only one sample for label:
    entity   node1 label  node2                              type_label  \
259   Q227  Q56061  P571   1991  'administrative territorial entity'@en   

     property_label    id lower_bound upper_bound  
259  '

/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/kneed/knee_locator.py:304: UserWarning: No knee/elbow found
  warnings.warn("No knee/elbow found")
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/kneed/knee_locator.py:218: RuntimeWarning: invalid value encountered in true_divide
  return (a - min(a)) / (max(a) - min(a))
/opt/anacon

only one sample for label:
       entity    node1 label  node2   type_label  property_label    id  \
473  Q5042950  Q836904  P571   1903  'brewer'@en  'inception'@en  E474   

    lower_bound upper_bound  
473                          

only one sample for label:
       entity   node1 label  node2    type_label  property_label    id  \
481  Q5374850  Q83405  P571   1837  'factory'@en  'inception'@en  E482   

    lower_bound upper_bound  
481                          

no knee found for these labels:
       entity    node1 label  node2    type_label  \
482  Q5374850  Q131734  P576   1991  'brewery'@en   
506   Q644625  Q131734  P576   2017  'brewery'@en   

                              property_label    id lower_bound upper_bound  
482  'dissolved, abolished or demolished'@en  E483                          
506  'dissolved, abolished or demolished'@en  E507                          
using median distance to k nearest neighbor instead (26.0)

only one sample for label:
       entity   

In [96]:
!head -50 $OUT/$NAME.entity_attribute_labels_time.year_bucketed.tsv | column -t -s $'\t'

entity     node1      label  node2  type_label                           property_label  id   lower_bound  upper_bound
Q1011      Q112099    P571   1975   'island nation'@en                   'inception'@en  E1   619.0
Q1011      Q3624078   P571   1975   'sovereign state'@en                 'inception'@en  E2   1615.5
Q1019      Q112099    P571   1960   'island nation'@en                   'inception'@en  E3   619.0
Q1019      Q3624078   P571   1960   'sovereign state'@en                 'inception'@en  E4   1615.5
Q1020773   Q3957      P571   1892   'town'@en                            'inception'@en  E5
Q1020773   Q902814    P571   1892   'border town'@en                     'inception'@en  E6
Q1027      Q112099    P571   1968   'island nation'@en                   'inception'@en  E7   619.0
Q1027      Q2221906   P571   1968   'geographic location'@en             'inception'@en  E8
Q1027      Q3624078   P571   1968   'sovereign state'@en                 'inception'@en  E9   1615.5
Q1

Aggregating distinct interval labels with positive entity counts

In [148]:
!kgtk query -i $OUT/$NAME.entity_attribute_labels_time.year_bucketed.tsv \
-o $OUT/$NAME.candidate_labels_ail_time.year_temp.tsv \
--graph-cache $STORE \
--match 'labels: (type)-[l {label:prop, property_label:lab, entity:e, lower_bound:lb, upper_bound:ub}]->(val)' \
--return 'type as type, prop as prop, lb as lower_bound, ub as upper_bound, count(e) as positives, lab as property_label, "_" as id' \
--order-by 'count(e) desc'

In [149]:
!kgtk rename-columns -i $OUT/$NAME.candidate_labels_ail_time.year_temp.tsv -o $OUT/$NAME.candidate_labels_ail_time.year_temp1.tsv \
--old-columns type prop lower_bound --new-columns node1 label node2 

In [150]:
!kgtk add-id -i $OUT/$NAME.candidate_labels_ail_time.year_temp1.tsv \
-o $OUT/$NAME.candidate_labels_ail_time.year.tsv --overwrite-id

In [151]:
!head -20 $OUT/$NAME.candidate_labels_ail_time.year.tsv | column -t -s $'\t'

node1      label  node2   upper_bound     positives       property_label  id
Q3624078   P571   1615.5  71              'inception'@en  E1
Q4830453   P571   1733.5  46              'inception'@en  E2
Q131734    P571   1922.5  20              'inception'@en  E3
Q51576574  P571   956.5   20              'inception'@en  E4
Q7270      P571   1635.5  19              'inception'@en  E5
Q131734    P571   1797.0  1922.5          16              'inception'@en  E6
Q179164    P571   1859.5  14              'inception'@en  E7
Q123480    P571   1529.5  13              'inception'@en  E8
Q3624078   P571   581.5   1370.5          12              'inception'@en  E9
Q112099    P571   619.0   11              'inception'@en  E10
Q6256      P571   1887.5  1940.0          11              'inception'@en  E11
Q6881511   P571   1905.5  11              'inception'@en  E12
Q167270    P571   1588.5  10              'inception'@en  E13
Q6881511   P571   1820.5  1905.5          10              'inception'@en  E14


In [142]:
avl_file_in = "{}/{}.entity_attribute_labels_quantity.tsv".format(os.environ["OUT"], os.environ["NAME"])
ail_file_out = "{}/{}.entity_attribute_labels_quantity_bucketed.tsv".format(os.environ["OUT"], os.environ["NAME"])
discretize_labels(avl_file_in, ail_file_out)

/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/kneed/knee_locator.py:304: UserWarning: No knee/elbow found
  warnings.warn("No knee/elbow found")


no knee found for these labels:
        entity  node1  label  node2 si_units wd_units type_label  \
0     Q1000597  Q3957  P1082  75074                    'town'@en   
1451  Q1020773  Q3957  P1082  64764                    'town'@en   

       property_label     id lower_bound upper_bound  
0     'population'@en     E1                          
1451  'population'@en  E1452                          
using median distance to k nearest neighbor instead (10310.0)

only one sample for label:
    entity    node1  label node2 si_units wd_units          type_label  \
337  Q1011  Q112099  P6897    87            Q11229  'island nation'@en   

         property_label    id lower_bound upper_bound  
337  'literacy rate'@en  E338                          

only one sample for label:
        entity  node1  label node2 si_units wd_units type_label  \
1452  Q1020773  Q3957  P2044   525            Q11573  'town'@en   

                      property_label     id lower_bound upper_bound  
1452  'elevati

/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned


only one sample for label:
     entity     node1  label node2 si_units wd_units  \
2080  Q1027  Q2221906  P2219   3.6            Q11229   

                    type_label                                property_label  \
2080  'geographic location'@en  'real gross domestic product growth rate'@en   

         id lower_bound upper_bound  
2080  E2081                          

only one sample for label:
     entity     node1  label node2 si_units wd_units  \
2131  Q1027  Q2221906  P2855    15            Q11229   

                    type_label property_label     id lower_bound upper_bound  
2131  'geographic location'@en  'VAT-rate'@en  E2132                          

only one sample for label:
     entity     node1  label node2 si_units wd_units  \
2132  Q1027  Q2221906  P2884   230            Q25250   

                    type_label      property_label     id lower_bound  \
2132  'geographic location'@en  'mains voltage'@en  E2133               

     upper_bound  
2132             

/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned


...

only one sample for label:
      entity    node1  label node2 si_units wd_units              type_label  \
80193   Q916  Q200464  P6591  43.5            Q25267  'Portuguese Empire'@en   

                        property_label      id lower_bound upper_bound  
80193  'maximum temperature record'@en  E80194                          

only one sample for label:
      entity    node1  label node2 si_units wd_units              type_label  \
80194   Q916  Q200464  P6897    66            Q11229  'Portuguese Empire'@en   

           property_label      id lower_bound upper_bound  
80194  'literacy rate'@en  E80195                          

no knee found for these labels:
          entity      node1  label node2 si_units wd_units       type_label  \
80547  Q93552342  Q15075508  P6088    20                    'beer brand'@en   
80561  Q93557205  Q15075508  P6088  29.5                    'beer brand'@en   
80573  Q93558270  Q15075508  P6088    20                    'beer brand'@en   
80587  Q

/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/kneed/knee_locator.py:304: UserWarning: No knee/elbow found
  warnings.warn("No knee/elbow found")
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/kneed/knee_locator.py:304: UserWarning: No knee/elbow found
  warnings.warn("No knee/elbow found")
/opt/anaconda3/envs/kgtk-env/lib/pytho

no knee found for these labels:
          entity    node1  label node2 si_units wd_units      type_label  \
80551  Q93552342  Q167270  P6088    20                    'trademark'@en   
80565  Q93557205  Q167270  P6088  29.5                    'trademark'@en   
80577  Q93558270  Q167270  P6088    20                    'trademark'@en   
80591  Q93559285  Q167270  P6088    15                    'trademark'@en   
80603  Q93560567  Q167270  P6088    26                    'trademark'@en   

             property_label      id lower_bound upper_bound  
80551  'beer bitterness'@en  E80552                          
80565  'beer bitterness'@en  E80566                          
80577  'beer bitterness'@en  E80578                          
80591  'beer bitterness'@en  E80592                          
80603  'beer bitterness'@en  E80604                          
using median distance to k nearest neighbor instead (3.5)

only one sample for label:
          entity    node1  label node2 si_units  wd_u

/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/kneed/knee_locator.py:304: UserWarning: No knee/elbow found
  warnings.warn("No knee/elbow found")
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/kneed/knee_locator.py:304: UserWarning: No knee/elbow found
  warnings.warn("No knee/elbow found")
/opt/anaconda3/envs/kgtk-env/lib/python3.7/site-packages/ipykernel_launcher.py:30: RankWarning: Polyfit may be poorly conditioned
/opt/anaconda3/envs/kgtk-env/lib/pytho

only one sample for label:
          entity    node1  label node2 si_units  wd_units       type_label  \
80584  Q93559285  Q131413  P2665   5.4           Q2080811  'wheat beer'@en   

               property_label      id lower_bound upper_bound  
80584  'alcohol by volume'@en  E80585                          

only one sample for label:
          entity    node1  label node2 si_units wd_units       type_label  \
80585  Q93559285  Q131413  P6088    15                    'wheat beer'@en   

             property_label      id lower_bound upper_bound  
80585  'beer bitterness'@en  E80586                          

only one sample for label:
          entity    node1  label node2 si_units  wd_units  type_label  \
80604  Q93560567  Q217825  P2665   5.5           Q2080811  'stout'@en   

               property_label      id lower_bound upper_bound  
80604  'alcohol by volume'@en  E80605                          

only one sample for label:
          entity    node1  label node2 si_units wd

In [144]:
!head -50 $OUT/$NAME.entity_attribute_labels_quantity_bucketed.tsv | column -t -s $'\t'

entity    node1    label  node2               si_units            wd_units                      type_label  property_label  id     lower_bound  upper_bound
Q1000597  Q3957    P1082  75074.0             'town'@en           'population'@en               E1
Q1011     Q112099  P1081  0.57                'island nation'@en  'Human Development Index'@en  E2          0.534           0.616
Q1011     Q112099  P1081  0.5720000000000001  'island nation'@en  'Human Development Index'@en  E3          0.534           0.616
Q1011     Q112099  P1081  0.575               'island nation'@en  'Human Development Index'@en  E4          0.534           0.616
Q1011     Q112099  P1081  0.585               'island nation'@en  'Human Development Index'@en  E5          0.534           0.616
Q1011     Q112099  P1081  0.589               'island nation'@en  'Human Development Index'@en  E6          0.534           0.616
Q1011     Q112099  P1081  0.5920000000000001  'island nation'@en  'Human Development Index'@en 

Aggregating distinct interval labels with positive entity counts

In [164]:
!kgtk query -i $OUT/$NAME.entity_attribute_labels_quantity_bucketed.tsv \
-o $OUT/$NAME.candidate_labels_ail_quantity_temp.tsv \
--graph-cache $STORE \
--match 'labels: (type)-[l {label:prop, property_label:lab, si_units:si, wd_units:wd, entity:e, lower_bound:lb, upper_bound:ub}]->(val)' \
--return 'type as type, prop as prop, si as si_units, wd as wd_units, lb as lower_bound, ub as upper_bound, count(e) as positives, lab as property_label, "_" as id' \
--order-by 'count(e) desc'

In [165]:
!kgtk rename-columns -i $OUT/$NAME.candidate_labels_ail_quantity_temp.tsv -o $OUT/$NAME.candidate_labels_ail_quantity_temp1.tsv \
--old-columns type prop lower_bound --new-columns node1 label node2 

In [166]:
!kgtk add-id -i $OUT/$NAME.candidate_labels_ail_quantity_temp1.tsv \
-o $OUT/$NAME.candidate_labels_ail_quantity.tsv --overwrite-id

In [167]:
!head -20 $OUT/$NAME.candidate_labels_ail_quantity.tsv | column -t -s $'\t'

node1     label  si_units    wd_units            node2                         upper_bound                   positives             property_label  id
Q3624078  P2131  Q4917       800249412297.1975   2752                          'nominal GDP'@en              E1
Q3624078  P1082  41699675.5  2689                'population'@en               E2
Q3624078  P2134  Q4917       67399246727.5       2529                          'total reserves'@en           E3
Q3624078  P2132  Q4917       21120.5             2424                          'nominal GDP per capita'@en   E4
Q3624078  P1081  0.595       0.9405000000000001  1893                          'Human Development Index'@en  E5
Q3624078  P1279  Q11229      -5.6                42.7                          1516                          'inflation rate'@en   E6
Q3624078  P4010  Q550207     1305343874679.5     1514                          'GDP (PPP)'@en                E7
Q3624078  P2299  Q550207     31357.3805          1463                     

### Combining entity --> attribute interval label mappings to single table

In [171]:
!kgtk cat \
-i $OUT/$NAME.entity_attribute_labels_quantity_bucketed.tsv \
-i $OUT/$NAME.entity_attribute_labels_time.year_bucketed.tsv \
-o $OUT/$NAME.entity_AILs_all.tsv

## 5. Create RALs with counts of positive entities

Here is an idea of what kind of values we should find (creating RALs from scratch for string attributes only)

In [626]:
!kgtk query -i $DATA/$NAME.part.wikibase-item.tsv -i $DATA/$NAME.part.string.tsv \
-i $OUT/$NAME.type_mapping.tsv -i $DATA/$NAME.label.en.tsv --graph-cache $STORE \
--match 'item: (n1)-[l1 {label:p1}]->(n2), type: (n1)-[]->(type1), label: (p1)-[:label]->(lab1), string: (n2)-[l2 {label:p2}]->(n3), type: (n2)-[]->(type2), label: (p2)-[:label]->(lab2)' \
--return 'distinct type1 as type1, p1 as prop1, type2 as type2, p2 as prop2, n3 as value, count(distinct n1) as positives, lab1 as prop1_label, lab2 as prop2_label' \
--where 'lab1.kgtk_lqstring_lang_suffix = "en"' \
--where 'lab2.kgtk_lqstring_lang_suffix = "en"' \
--order-by 'count(distinct n1) desc' \
--limit 5 \
| column -t -s $'\t'

type1     prop1  type2     prop2  value           positives  prop1_label               prop2_label
Q131734   P452   Q8148     P373   "Beer brewing"  77         'industry'@en             'Commons category'@en
Q3624078  P530   Q3624078  P3238  "0"             68         'diplomatic relation'@en  'trunk prefix'@en
Q3624078  P530   Q4209223  P3238  "0"             68         'diplomatic relation'@en  'trunk prefix'@en
Q3624078  P530   Q43702    P3238  "0"             68         'diplomatic relation'@en  'trunk prefix'@en
Q3624078  P530   Q619610   P3238  "0"             68         'diplomatic relation'@en  'trunk prefix'@en


**NOTE:** It looks like having all of the entity --> labels mappings together in one file would be nice here. I don't think there is a need to keep these separate? Maybe keep AVL and AIL separate.

Now using the REL table and the entities --> attribute labels table that we built in steps 2 and 4. Also keep track of counts of positive entities for each label

Trying to reuse RELs, however, when counting positives, we would need to sum each num_pos that matches the line - not sure how to do this, so the below won't capture when type1 --> x1 and type1 --> x2 resolve to be the same label: i.e. type1 --> typex --> val. See further down for alternate solution.

In [624]:
!kgtk query -i $OUT/$NAME.candidate_labels_rel_item.tsv -i $OUT/$NAME.entity_attribute_labels_string.tsv \
--graph-cache $STORE \
--match 'rel: (t1)-[l1 {label:p1, positives:num_pos}]->(v1), entity_attribute: (t2)-[l {entity:v1, label:p2}]->(v2)' \
--return 't1 as type, p1 as prop, t2 as value_type, p2 as value_prop, v2 as value_val, num_pos as positives, "_" as id' \
--order-by "kgtk_quantity_number_int(num_pos) desc" \
--limit 5 \
| column -t -s $'\t'

type      prop  value_type  value_prop  value_val       positives  id
Q131734   P452  Q8148       P373        "Beer brewing"  77         _
Q3624078  P530  Q3624078    P2258       "262"           67         _
Q3624078  P530  Q3624078    P2572       "Deutschland"   67         _
Q3624078  P530  Q3624078    P2979       "211"           67         _
Q3624078  P530  Q3624078    P2979       "218"           67         _


This way should work.

In [625]:
!kgtk query -i $DATA/$NAME.part.wikibase-item.tsv -i $OUT/$NAME.type_mapping.tsv \
-i $OUT/$NAME.entity_attribute_labels_string.tsv --graph-cache $STORE \
--match 'item: (n1)-[l1 {label:p1}]->(n2), type: (n1)-[]->(t1), entity_attribute: (t2)-[l2 {label:p2, entity:n2}]->(val)' \
--return 't1 as type1, p1 as prop1, t2 as type2, p2 as prop2, val as value, count(distinct n1) as positives, "_" as id' \
--order-by "count(distinct n1) desc" \
--limit 5 \
| column -t -s $'\t'

type1     prop1  type2     prop2  value           positives  id
Q131734   P452   Q8148     P373   "Beer brewing"  77         _
Q3624078  P530   Q3624078  P3238  "0"             68         _
Q3624078  P530   Q4209223  P3238  "0"             68         _
Q3624078  P530   Q43702    P3238  "0"             68         _
Q3624078  P530   Q619610   P3238  "0"             68         _


And now doing this for all attribute types:

attribute value labels:

In [178]:
!kgtk query -i $DATA/$NAME.part.wikibase-item.tsv -i $OUT/$NAME.type_mapping.tsv -i $DATA/$NAME.label.en.tsv \
-i $OUT/$NAME.entity_AVLs_all.tsv -o $OUT/$NAME.candidate_labels_ravl_temp.tsv --graph-cache $STORE \
--match 'item: (n1)-[l1 {label:p1}]->(n2), type: (n1)-[]->(t1), entity_AVLs: (t2)-[l2 {label:p2, entity:n2}]->(val), label: (p2)-[:label]->(lab2)' \
--return 't1 as type1, p1 as prop1, t2 as type2, p2 as prop2, lab2 as prop2_label, val as value, count(distinct n1) as positives, "_" as id' \
--order-by "count(distinct n1) desc" \
--where 'lab2.kgtk_lqstring_lang_suffix = "en"'

In [179]:
!kgtk rename-columns -i $OUT/$NAME.candidate_labels_ravl_temp.tsv -o $OUT/$NAME.candidate_labels_ravl_temp1.tsv \
--old-columns type1 prop1 type2 --new-columns node1 label node2 

In [180]:
!kgtk add-id -i $OUT/$NAME.candidate_labels_ravl_temp1.tsv \
-o $OUT/$NAME.candidate_labels_ravl.tsv --overwrite-id

In [181]:
!head $OUT/$NAME.candidate_labels_ravl.tsv | column -t -s $'\t'

node1     label  node2     prop2  prop2_label                   value           positives  id
Q131734   P452   Q8148     P373   'Commons category'@en         "Beer brewing"  77         E1
Q131734   P452   Q8148     P580   'start time'@en               -3500           77         E2
Q3624078  P530   Q3624078  P1081  'Human Development Index'@en  0.801           68         E3
Q3624078  P530   Q3624078  P1081  'Human Development Index'@en  0.809           68         E4
Q3624078  P530   Q3624078  P1081  'Human Development Index'@en  0.814           68         E5
Q3624078  P530   Q3624078  P1081  'Human Development Index'@en  0.824           68         E6
Q3624078  P530   Q3624078  P1081  'Human Development Index'@en  0.829           68         E7
Q3624078  P530   Q3624078  P1081  'Human Development Index'@en  0.83            68         E8
Q3624078  P530   Q3624078  P1081  'Human Development Index'@en  0.834           68         E9


attribute interval labels:

In [173]:
!kgtk query -i $DATA/$NAME.part.wikibase-item.tsv -i $OUT/$NAME.type_mapping.tsv -i $DATA/$NAME.label.en.tsv \
-i $OUT/$NAME.entity_AILs_all.tsv -o $OUT/$NAME.candidate_labels_rail_temp.tsv --graph-cache $STORE \
--match 'item: (n1)-[l1 {label:p1}]->(n2), type: (n1)-[]->(t1), entity_AILs: (t2)-[l2 {label:p2, entity:n2, lower_bound:lb, upper_bound:ub, wd_units:wd, si_units:si}]->(val), label: (p2)-[:label]->(lab2)' \
--return 't1 as type1, p1 as prop1, t2 as type2, p2 as prop2, lab2 as prop2_label, si as si_units, wd as wd_units, lb as lower_bound, ub as upper_bound, count(distinct n1) as positives, "_" as id' \
--order-by "count(distinct n1) desc" \
--where 'lab2.kgtk_lqstring_lang_suffix = "en"'

In [174]:
!kgtk rename-columns -i $OUT/$NAME.candidate_labels_rail_temp.tsv -o $OUT/$NAME.candidate_labels_rail_temp1.tsv \
--old-columns type1 prop1 type2 --new-columns node1 label node2 

In [175]:
!kgtk add-id -i $OUT/$NAME.candidate_labels_rail_temp1.tsv \
-o $OUT/$NAME.candidate_labels_rail.tsv --overwrite-id

In [176]:
!head $OUT/$NAME.candidate_labels_rail.tsv | column -t -s $'\t'

node1    label  node2     prop2  prop2_label                                   si_units  wd_units            lower_bound  upper_bound  positives  id
Q131734  P452   Q8148     P580   'start time'@en                               77        E1
Q131734  P17    Q3624078  P1081  'Human Development Index'@en                  0.595     0.9405000000000001  70           E2
Q131734  P17    Q3624078  P1279  'inflation rate'@en                           Q11229    -5.6                42.7         70           E3
Q131734  P17    Q3624078  P2131  'nominal GDP'@en                              Q4917     800249412297.1975   70           E4
Q131734  P17    Q3624078  P2219  'real gross domestic product growth rate'@en  Q11229    -4.609999999999999  70           E5
Q131734  P17    Q3624078  P2250  'life expectancy'@en                          Q577      70.059205           83.084145    70           E6

Q131734  P17    Q3624078  P2132  'nominal GDP per capita'@en                   Q4917     21120.5           